# New Style HDMI input and Pixel Formatting

This notebook introduces the new features of PYNQ 2.0 for interacting with the video pipeline. The API has been completely
redesigned with high performance image processing applications in mind.

To start, download the base overlay and instantiate the HDMI input and output.

BLOCK 1 (Prepare HDMI - should see raw output on monitor after running this)

In [ ]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *

base = BaseOverlay("hls.bit")
hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out


# config hdmi
import copy
hdmi_in.configure()
hdmi_out_mode = copy.copy(hdmi_in.mode)
print(hdmi_in.mode)
print(hdmi_out_mode)

hdmi_out_mode.width = 1920
hdmi_out_mode.height = 1080
hdmi_out_mode.stride = 1920 * 3
hdmi_out_mode.fps = 60
hdmi_out.configure(hdmi_out_mode)

print(hdmi_out_mode.stride)

hdmi_in.start()
hdmi_out.start()

hdmi_in.tie(hdmi_out)


# start dma
from pynq import MMIO
import numpy as np
from pynq import allocate
print(hex(base.ip_dict['axi_dma_0']['phys_addr']))
# please confirm the address of dma is set  correctly
axi_dma = MMIO(hex(base.ip_dict['axi_dma_0']['phys_addr']), 0x60)

# Read the status and control logic of dma
# print(hex(axi_dma.read(0)))     #MM2S control
# print(hex(axi_dma.read(4)))     #MM2S status

# print(hex(axi_dma.read(0x30)))  #S2MM control
# print(hex(axi_dma.read(0x34)))  #S2MM status

BLOCK2 (Start streaming)

In [5]:
_4k_byte = 3840 * 2160 * 3
_1k_byte = 1920 * 1080 * 3

f = hdmi_in.readframe()
output_buffer = allocate(shape=(1080, 1920, 3),dtype=np.uint8)
# write
axi_dma.write(0, 1)
axi_dma.write(0x1c, 0)
axi_dma.write(0x18, f.device_address)
axi_dma.write(0x28, _4k_byte)

# read
axi_dma.write(0x30, 1)
axi_dma.write(0x4c, 0)
axi_dma.write(0x48, output_buffer.device_address)
axi_dma.write(0x58, _1k_byte)

while hex(axi_dma.read(4)) != "0x1002":
              pass


import time
num_frames = 1200
start = time.time()
for _ in range(num_frames):
    f = hdmi_in.readframe()
    
    while hex(axi_dma.read(4)) != "0x1002":
              pass
    # write
    axi_dma.write(0, 1)
    axi_dma.write(0x1c, 0)
    axi_dma.write(0x18, f.device_address)
    axi_dma.write(0x28, _4k_byte)
    
    while hex(axi_dma.read(0x34)) != "0x1002":
              pass
        
        
    hdmi_out.writeframe(output_buffer)
    

    # read
    axi_dma.write(0x30, 1)
    axi_dma.write(0x4c, 0)
    axi_dma.write(0x48, output_buffer.device_address)
    axi_dma.write(0x58, _1k_byte)


    
    
end = time.time()
print("Frames per second:  " + str(num_frames / (end - start)))

BLOCK3 (Save board output)

In [ ]:
import PIL.Image
import numpy as np
img = PIL.Image.fromarray(output_buffer)
img.save("board.png")
np.save("board.npy", output_buffer)
img

BLOCK4 (Close HDMI)

In [13]:
hdmi_out.close()
hdmi_in.close()